In [1]:
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [2]:
CLIENT_ID = 'GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR' # your Foursquare ID
CLIENT_SECRET = 'MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR
CLIENT_SECRET:MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3


In [3]:
#source from: https://en.wikipedia.org/wiki/Boroughs_of_New_York_City

from collections import OrderedDict
stac = {'Borough':["Bronx", "Brooklyn", "Manhattan", "Queens", "Staten Island"],
        'Population':[1471160, 2648771, 1664727, 2358582, 479458],
        'GDP':[28.787, 63.303, 629.682, 73.842, 11.249],
        'per capita':[19570, 23900, 378250, 31310, 23460],
        'square miles':[42.1, 70.82, 22.83, 108.53, 58.37],
        'persons /sq.mi':[34653, 37137, 72033, 21460, 8112]}
df_ny = pd.DataFrame.from_dict(stac)

In [4]:
df_ny

,Borough,Population,GDP,per capita,square miles,persons /sq.mi
0,Bronx,1471160,28.787,19570,42.10,34653
1,Brooklyn,2648771,63.303,23900,70.82,37137
2,Manhattan,1664727,629.682,378250,22.83,72033
3,Queens,2358582,73.842,31310,108.53,21460
4,Staten Island,479458,11.249,23460,58.37,8112


In [5]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [6]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [7]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [8]:
neighborhoods_data = newyork_data['features']

In [9]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [10]:
#neighborhoods

In [11]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [12]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [13]:
df_nygeo = pd.merge(neighborhoods, df_ny, on='Borough', how = 'left')
df_nygeo.shape

(306, 9)

In [14]:
brooklyn_df = df_nygeo.loc[df_nygeo.Borough=="Brooklyn"].reset_index(drop=True)
brooklyn_df.shape

(70, 9)

In [15]:
brooklyn_df.head()

,Borough,Neighborhood,Latitude,Longitude,Population,GDP,per capita,square miles,persons /sq.mi
0,Brooklyn,Bay Ridge,40.625801,-74.030621,2648771,63.303,23900,70.82,37137
1,Brooklyn,Bensonhurst,40.611009,-73.995180,2648771,63.303,23900,70.82,37137
2,Brooklyn,Sunset Park,40.645103,-74.010316,2648771,63.303,23900,70.82,37137
3,Brooklyn,Greenpoint,40.730201,-73.954241,2648771,63.303,23900,70.82,37137
4,Brooklyn,Gravesend,40.595260,-73.973471,2648771,63.303,23900,70.82,37137


In [16]:
address_1 = 'Brooklyn, NY'

geolocator_1 = Nominatim(user_agent="br_explorer")
location_1 = geolocator_1.geocode(address_1)
latitude_1 = location_1.latitude
longitude_1 = location_1.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude_1, longitude_1))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [17]:
# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude_1, longitude_1], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_df['Latitude'], brooklyn_df['Longitude'], brooklyn_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

In [18]:
search_query = 'restaurant' 
radius = 5000
print(search_query + ' .... OK!')

restaurant .... OK!


In [19]:
url_1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_1, longitude_1, VERSION, search_query, radius, LIMIT)
url_1

'https://api.foursquare.com/v2/venues/search?client_id=GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR&client_secret=MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3&ll=40.6501038,-73.9495823&v=20180604&query=restaurant&radius=5000&limit=300'

In [20]:
results_1 = requests.get(url_1).json()
results_1

{'meta': {'code': 200, 'requestId': '5d3b3e1792e7a9002cc9d999'},
 'response': {'venues': [{'id': '50b91d3de4b0802eb8488f39',
    'name': 'Kreyol Flavor Bakery & Restaurant',
    'location': {'address': '2816 Church Ave',
     'lat': 40.650820092517755,
     'lng': -73.95077705383301,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.650820092517755,
       'lng': -73.95077705383301}],
     'distance': 128,
     'postalCode': '11226',
     'cc': 'US',
     'city': 'Brooklyn',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['2816 Church Ave',
      'Brooklyn, NY 11226',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d144941735',
      'name': 'Caribbean Restaurant',
      'pluralName': 'Caribbean Restaurants',
      'shortName': 'Caribbean',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caribbean_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1564163607',
    'hasPerk': False

In [21]:
# assign relevant part of JSON to venues
venues_1 = results_1['response']['venues']

# tranform venues into a dataframe
dataframe_1 = json_normalize(venues_1)
dataframe_1.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,50b91d3de4b0802eb8488f39,Kreyol Flavor Bakery & Restaurant,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",v-1564163607,False,2816 Church Ave,40.650820,-73.950777,"[{'label': 'display', 'lat': 40.65082009251775...",128,11226,US,Brooklyn,NY,United States,"[2816 Church Ave, Brooklyn, NY 11226, United S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3fd66200f964a5207cf11ee3,Junior's Restaurant,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",v-1564163607,False,386 Flatbush Avenue Ext,40.690011,-73.981734,"[{'label': 'display', 'lat': 40.69001145156075...",5206,11201,US,Brooklyn,NY,United States,"[386 Flatbush Avenue Ext (at DeKalb Ave), Broo...",at DeKalb Ave,Downtown Brooklyn,291315,https://www.seamless.com/menu/juniors-restaura...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,66852513
2,4bdcade72a3a0f470fecb3b6,MangoSeed Restaurant,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",v-1564163607,False,757 Flatbush Ave,40.654403,-73.959560,"[{'label': 'display', 'lat': 40.65440267666848...",969,11226,US,Brooklyn,NY,United States,"[757 Flatbush Ave (Between Lenox & Clarkson), ...",Between Lenox & Clarkson,NaN,108162,https://www.seamless.com/menu/mangoseed-757-fl...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,39496036
3,4bfafc771134b713e48c0fc9,Green Lake Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1564163607,False,795 Flatbush Ave,40.653110,-73.959343,"[{'label': 'display', 'lat': 40.65311049787105...",889,11226,US,Brooklyn,NY,United States,"[795 Flatbush Ave (btw Lenox & Caton), Brookly...",btw Lenox & Caton,NaN,23279,https://www.seamless.com/menu/green-lake-resta...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN
4,4a6c86e5f964a520b8d01fe3,Tom's Restaurant,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",v-1564163607,False,782 Washington Ave,40.674537,-73.963216,"[{'label': 'display', 'lat': 40.67453698094471...",2953,11238,US,Brooklyn,NY,United States,"[782 Washington Ave (at Sterling Pl), Brooklyn...",at Sterling Pl,Prospect Heights,297390,https://www.seamless.com/menu/toms-restaurant-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,82422922


In [22]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_1 = ['name', 'categories'] + [col for col in dataframe_1.columns if col.startswith('location.')] + ['id']
dataframe_filtered_1 = dataframe_1.loc[:, filtered_columns_1]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_1['categories'] = dataframe_filtered_1.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_1.columns = [column_1.split('.')[-1] for column_1 in dataframe_filtered_1.columns]

dataframe_filtered_1

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Kreyol Flavor Bakery & Restaurant,Caribbean Restaurant,2816 Church Ave,40.650820,-73.950777,"[{'label': 'display', 'lat': 40.65082009251775...",128,11226,US,Brooklyn,NY,United States,"[2816 Church Ave, Brooklyn, NY 11226, United S...",NaN,NaN,50b91d3de4b0802eb8488f39
1,Junior's Restaurant,Diner,386 Flatbush Avenue Ext,40.690011,-73.981734,"[{'label': 'display', 'lat': 40.69001145156075...",5206,11201,US,Brooklyn,NY,United States,"[386 Flatbush Avenue Ext (at DeKalb Ave), Broo...",at DeKalb Ave,Downtown Brooklyn,3fd66200f964a5207cf11ee3
2,MangoSeed Restaurant,Caribbean Restaurant,757 Flatbush Ave,40.654403,-73.959560,"[{'label': 'display', 'lat': 40.65440267666848...",969,11226,US,Brooklyn,NY,United States,"[757 Flatbush Ave (Between Lenox & Clarkson), ...",Between Lenox & Clarkson,NaN,4bdcade72a3a0f470fecb3b6
3,Green Lake Chinese Restaurant,Chinese Restaurant,795 Flatbush Ave,40.653110,-73.959343,"[{'label': 'display', 'lat': 40.65311049787105...",889,11226,US,Brooklyn,NY,United States,"[795 Flatbush Ave (btw Lenox & Caton), Brookly...",btw Lenox & Caton,NaN,4bfafc771134b713e48c0fc9
4,Tom's Restaurant,Diner,782 Washington Ave,40.674537,-73.963216,"[{'label': 'display', 'lat': 40.67453698094471...",2953,11238,US,Brooklyn,NY,United States,"[782 Washington Ave (at Sterling Pl), Brooklyn...",at Sterling Pl,Prospect Heights,4a6c86e5f964a520b8d01fe3
5,Siu's Restaurant,Chinese Restaurant,4324 Church Ave,40.651404,-73.936209,"[{'label': 'display', 'lat': 40.65140401072426...",1138,11203,US,Brooklyn,NY,United States,"[4324 Church Ave, Brooklyn, NY 11203, United S...",NaN,NaN,4d3b0c6d1fc36dcbbc206df5
6,C & J Jamaican Restaurant,Caribbean Restaurant,817 Utica Ave,40.653232,-73.930493,"[{'label': 'display', 'lat': 40.65323183619257...",1649,11203,US,Brooklyn,NY,United States,"[817 Utica Ave (at Linden Blvd), Brooklyn, NY ...",at Linden Blvd,NaN,4bc4c577abf49521628bc593
7,Breeze's Jamaican Restaurant,Caribbean Restaurant,1135 Flatbush Ave,40.642598,-73.957644,"[{'label': 'display', 'lat': 40.64259828580389...",1077,11226,US,Brooklyn,NY,United States,"[1135 Flatbush Ave, Brooklyn, NY 11226, United...",NaN,NaN,4e33140eb0fb17f64f7edaac
8,Madina Restaurant and Sweets,Indian Restaurant,563 Coney Island Ave,40.643697,-73.969999,"[{'label': 'display', 'lat': 40.64369687020608...",1866,11218,US,Brooklyn,NY,United States,"[563 Coney Island Ave (Beverly Rd & Lewis Pl),...",Beverly Rd & Lewis Pl,NaN,4b718914f964a520c04b2de3
9,China Hao Restaurant,Chinese Restaurant,937 Utica Ave,40.650186,-73.930144,"[{'label': 'display', 'lat': 40.65018596306685...",1641,11203,US,Brooklyn,NY,United States,"[937 Utica Ave (Snyder Ave), Brooklyn, NY 1120...",Snyder Ave,NaN,4c1ad571eac020a10a0144c2


In [27]:
dataframe_filtered_1.shape

(50, 16)

In [28]:
venues_map_1 = folium.Map(location=[latitude_1, longitude_1], zoom_start=11) 

# add a red circle marker to represent the Brooklyn
folium.features.CircleMarker(
    [latitude_1, longitude_1],
    radius=10,
    color='red',
    popup='Brooklyn',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map_1)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_1.lat, dataframe_filtered_1.lng, dataframe_filtered_1.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_1)

# display map
venues_map_1

In [29]:
#categories count
dataframe_filtered_1.groupby('categories').count()

,name,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
categories,,,,,,,,,,,,,,,
African Restaurant,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Asian Restaurant,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Bakery,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
Bar,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Caribbean Restaurant,17,15,17,17,17,17,17,17,17,17,17,17,10,1,17
Chinese Restaurant,9,8,9,9,9,9,9,9,9,9,9,9,7,0,9
Diner,5,5,5,5,5,5,5,5,5,5,5,5,4,2,5
Food,2,2,2,2,2,2,2,2,2,2,2,2,0,0,2
Halal Restaurant,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1


In [30]:
if len(results_1['response']['venues']) == 0:
    trending_venues_1_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues_1 = results_1['response']['venues']
    trending_venues_1_df = json_normalize(trending_venues_1)

    # filter columns
    columns_filtered_1 = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_1_df = trending_venues_1_df.loc[:, columns_filtered_1]

    # filter the category for each row
    trending_venues_1_df['categories'] = trending_venues_1_df.apply(get_category_type, axis=1)

In [31]:
trending_venues_1_df

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,Kreyol Flavor Bakery & Restaurant,Caribbean Restaurant,128,Brooklyn,11226,NY,United States,40.650820,-73.950777
1,Junior's Restaurant,Diner,5206,Brooklyn,11201,NY,United States,40.690011,-73.981734
2,MangoSeed Restaurant,Caribbean Restaurant,969,Brooklyn,11226,NY,United States,40.654403,-73.959560
3,Green Lake Chinese Restaurant,Chinese Restaurant,889,Brooklyn,11226,NY,United States,40.653110,-73.959343
4,Tom's Restaurant,Diner,2953,Brooklyn,11238,NY,United States,40.674537,-73.963216
5,Siu's Restaurant,Chinese Restaurant,1138,Brooklyn,11203,NY,United States,40.651404,-73.936209
6,C & J Jamaican Restaurant,Caribbean Restaurant,1649,Brooklyn,11203,NY,United States,40.653232,-73.930493
7,Breeze's Jamaican Restaurant,Caribbean Restaurant,1077,Brooklyn,11226,NY,United States,40.642598,-73.957644
8,Madina Restaurant and Sweets,Indian Restaurant,1866,Brooklyn,11218,NY,United States,40.643697,-73.969999
9,China Hao Restaurant,Chinese Restaurant,1641,Brooklyn,11203,NY,United States,40.650186,-73.930144


In [32]:
if len(results_1['response']['venues']) == 0:
    trending_venues_map_1 = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    trending_venues_map_1 = folium.Map(location=[latitude_1, longitude_1], zoom_start=15) # generate map centred around Brooklyn


    # add Brooklyn as a red circle mark
    folium.features.CircleMarker(
        [latitude_1, longitude_1],
        radius=10,
        popup='Brooklyn',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(trending_venues_map_1)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_1_df['location.lat'], trending_venues_1_df['location.lng'], trending_venues_1_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(trending_venues_map_1)
        
        
 

In [33]:
# display map
trending_venues_map_1 

In [ ]:
#same process for Queens

In [34]:
queens_df = df_nygeo.loc[df_nygeo.Borough=="Queens"].reset_index(drop=True)
queens_df.shape

(81, 9)

In [35]:
queens_df.head()

,Borough,Neighborhood,Latitude,Longitude,Population,GDP,per capita,square miles,persons /sq.mi
0,Queens,Astoria,40.768509,-73.915654,2358582,73.842,31310,108.53,21460
1,Queens,Woodside,40.746349,-73.901842,2358582,73.842,31310,108.53,21460
2,Queens,Jackson Heights,40.751981,-73.882821,2358582,73.842,31310,108.53,21460
3,Queens,Elmhurst,40.744049,-73.881656,2358582,73.842,31310,108.53,21460
4,Queens,Howard Beach,40.654225,-73.838138,2358582,73.842,31310,108.53,21460


In [36]:
address_2 = 'Queens, NY'

geolocator_2 = Nominatim(user_agent="qu_explorer")
location_2 = geolocator_2.geocode(address_2)
latitude_2 = location_2.latitude
longitude_2 = location_2.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude_2, longitude_2))

The geograpical coordinate of Queens are 40.6524927, -73.7914214158161.


In [37]:
# create map of Queens using latitude and longitude values
map_queens = folium.Map(location=[latitude_2, longitude_2], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(queens_df['Latitude'], queens_df['Longitude'], queens_df['Borough'], queens_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

In [38]:
search_query = 'restaurant' 
radius = 5000
print(search_query + ' .... OK!')

restaurant .... OK!


In [39]:
url_2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_2, longitude_2, VERSION, search_query, radius, LIMIT)
url_2

'https://api.foursquare.com/v2/venues/search?client_id=GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR&client_secret=MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3&ll=40.6524927,-73.7914214158161&v=20180604&query=restaurant&radius=5000&limit=300'

In [40]:
results_2 = requests.get(url_2).json()
results_2

{'meta': {'code': 200, 'requestId': '5d3b3f3b8ad62e0039521326'},
 'response': {'venues': [{'id': '53e7bd7e498e1357c2487fa7',
    'name': '141 Restaurant - Hilton New York JFK',
    'location': {'lat': 40.66743276921117,
     'lng': -73.7958065952312,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.66743276921117,
       'lng': -73.7958065952312}],
     'distance': 1703,
     'cc': 'US',
     'city': 'Queens',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['Queens, NY', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1c4941735',
      'name': 'Restaurant',
      'pluralName': 'Restaurants',
      'shortName': 'Restaurant',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1564163900',
    'hasPerk': False},
   {'id': '4e4e3d96bd4101d0d7a5b41a',
    'name': 'Panda Chinese Restaurant',
    'location': {'address': '14611 Rockaway 

In [41]:
# assign relevant part of JSON to venues
venues_2 = results_2['response']['venues']

# tranform venues into a dataframe
dataframe_2 = json_normalize(venues_2)
dataframe_2.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.postalCode,location.crossStreet,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,53e7bd7e498e1357c2487fa7,141 Restaurant - Hilton New York JFK,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1564163900,False,40.667433,-73.795807,"[{'label': 'display', 'lat': 40.66743276921117...",1703,US,Queens,NY,United States,"[Queens, NY, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4e4e3d96bd4101d0d7a5b41a,Panda Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1564163900,False,40.673527,-73.791046,"[{'label': 'display', 'lat': 40.67352651585146...",2341,US,Jamaica,NY,United States,"[14611 Rockaway Blvd, Jamaica, NY 11436, Unite...",14611 Rockaway Blvd,11436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4d793f73132d8eec4f7815be,G's Restaurant and Bakery,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",v-1564163900,False,40.674358,-73.802274,"[{'label': 'display', 'lat': 40.674358, 'lng':...",2600,US,South Ozone Park,NY,United States,"[13536 Rockaway Blvd, South Ozone Park, NY 114...",13536 Rockaway Blvd,11420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4bdf32a3e75c0f47973cca03,Port Royal Jamaican Restaurant,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",v-1564163900,False,40.686740,-73.794102,"[{'label': 'display', 'lat': 40.68673992263182...",3819,US,Jamaica,NY,United States,"[11307 Sutphin Blvd (at Linden Blvd), Jamaica,...",11307 Sutphin Blvd,11435,at Linden Blvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4c64a7ce94160f473d872d17,Vetro Restaurant & Lounge,"[{'id': '4bf58dd8d48988d11a941735', 'name': 'O...",v-1564163900,False,40.649237,-73.837212,"[{'label': 'display', 'lat': 40.64923729597181...",3884,US,Howard Beach,NY,United States,"[164-49 Crossbay Blvd (at 165th Ave.), Howard ...",164-49 Crossbay Blvd,11414,at 165th Ave.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_2 = ['name', 'categories'] + [col for col in dataframe_2.columns if col.startswith('location.')] + ['id']
dataframe_filtered_2 = dataframe_2.loc[:, filtered_columns_2]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_2['categories'] = dataframe_filtered_2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_2.columns = [column_2.split('.')[-1] for column_2 in dataframe_filtered_2.columns]

dataframe_filtered_2

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,postalCode,crossStreet,neighborhood,id
0,141 Restaurant - Hilton New York JFK,Restaurant,40.667433,-73.795807,"[{'label': 'display', 'lat': 40.66743276921117...",1703,US,Queens,NY,United States,"[Queens, NY, United States]",NaN,NaN,NaN,NaN,53e7bd7e498e1357c2487fa7
1,Panda Chinese Restaurant,Chinese Restaurant,40.673527,-73.791046,"[{'label': 'display', 'lat': 40.67352651585146...",2341,US,Jamaica,NY,United States,"[14611 Rockaway Blvd, Jamaica, NY 11436, Unite...",14611 Rockaway Blvd,11436,NaN,NaN,4e4e3d96bd4101d0d7a5b41a
2,G's Restaurant and Bakery,Caribbean Restaurant,40.674358,-73.802274,"[{'label': 'display', 'lat': 40.674358, 'lng':...",2600,US,South Ozone Park,NY,United States,"[13536 Rockaway Blvd, South Ozone Park, NY 114...",13536 Rockaway Blvd,11420,NaN,NaN,4d793f73132d8eec4f7815be
3,Port Royal Jamaican Restaurant,Caribbean Restaurant,40.686740,-73.794102,"[{'label': 'display', 'lat': 40.68673992263182...",3819,US,Jamaica,NY,United States,"[11307 Sutphin Blvd (at Linden Blvd), Jamaica,...",11307 Sutphin Blvd,11435,at Linden Blvd,NaN,4bdf32a3e75c0f47973cca03
4,Vetro Restaurant & Lounge,Other Nightlife,40.649237,-73.837212,"[{'label': 'display', 'lat': 40.64923729597181...",3884,US,Howard Beach,NY,United States,"[164-49 Crossbay Blvd (at 165th Ave.), Howard ...",164-49 Crossbay Blvd,11414,at 165th Ave.,NaN,4c64a7ce94160f473d872d17
5,Aldo's Pizzeria & Restaurant,Italian Restaurant,40.672073,-73.843016,"[{'label': 'display', 'lat': 40.67207259744984...",4871,US,Ozone Park,NY,United States,"[137-01 Crossbay Blvd. (at Pitkin Ave.), Ozone...",137-01 Crossbay Blvd.,11417,at Pitkin Ave.,Ozone Park,4cb21cebdb32f04db2a7cd4d
6,BJ's Restaurant & Brewhouse,American Restaurant,40.663520,-73.725640,"[{'label': 'display', 'lat': 40.66351977391455...",5689,US,Valley Stream,NY,United States,"[750 Sunrise Hwy 5 Green Acres Common, Valley ...",750 Sunrise Hwy 5 Green Acres Common,11581,NaN,NaN,57edb457498e1dd0853c64fa
7,Ho Wan Restaurant,Chinese Restaurant,40.684857,-73.830962,"[{'label': 'display', 'lat': 40.68485730394045...",4911,US,Jamaica,NY,United States,"[11105 Liberty Ave #A, Jamaica, NY 11419, Unit...",11105 Liberty Ave #A,11419,NaN,NaN,4bfedc9bc30a2d7f1e1e101d
8,The Nest Restaurant,Caribbean Restaurant,40.691697,-73.820287,"[{'label': 'display', 'lat': 40.69169745908868...",4998,US,Richmond Hill,NY,United States,"[12517 101st Ave (at 126th St), Richmond Hill,...",12517 101st Ave,11419,at 126th St,NaN,4c1ec7e5fcf8c9b6c2d8ad0b
9,The Palm Bar & Grille,Steakhouse,40.643562,-73.782897,"[{'label': 'display', 'lat': 40.64356153548163...",1227,US,Jamaica,NY,United States,"[Terminal 4 (JFK International Airport), Jamai...",Terminal 4,11430,JFK International Airport,NaN,4b6b5520f964a52077012ce3


In [44]:
venues_map_2 = folium.Map(location=[latitude_2, longitude_2], zoom_start=11) # generate map centred around Queens

# add a red circle marker to represent Queens
folium.features.CircleMarker(
    [latitude_2, longitude_2],
    radius=10,
    color='red',
    popup='Queens',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map_2)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_2.lat, dataframe_filtered_2.lng, dataframe_filtered_2.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_2)

# display map
venues_map_2

In [43]:
dataframe_filtered_2.shape

(50, 16)

In [45]:
dataframe_filtered_2.groupby('categories').count()

,name,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,postalCode,crossStreet,neighborhood,id
categories,,,,,,,,,,,,,,,
American Restaurant,3,3,3,3,3,3,3,3,3,3,1,3,0,0,3
Asian Restaurant,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1
Bakery,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Caribbean Restaurant,13,13,13,13,13,13,13,13,13,13,13,13,3,0,13
Chinese Restaurant,8,8,8,8,8,8,7,8,8,8,7,7,0,0,8
Food,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Greek Restaurant,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Italian Restaurant,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2
Latin American Restaurant,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1


In [46]:
if len(results_2['response']['venues']) == 0:
    trending_venues_2_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues_2 = results_2['response']['venues']
    trending_venues_2_df = json_normalize(trending_venues_2)

    # filter columns
    columns_filtered_2 = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_2_df = trending_venues_2_df.loc[:, columns_filtered_2]

    # filter the category for each row
    trending_venues_2_df['categories'] = trending_venues_2_df.apply(get_category_type, axis=1)

In [50]:
# display trending venues
trending_venues_2_df

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,141 Restaurant - Hilton New York JFK,Restaurant,1703,Queens,NaN,NY,United States,40.667433,-73.795807
1,Panda Chinese Restaurant,Chinese Restaurant,2341,Jamaica,11436,NY,United States,40.673527,-73.791046
2,G's Restaurant and Bakery,Caribbean Restaurant,2600,South Ozone Park,11420,NY,United States,40.674358,-73.802274
3,Vetro Restaurant & Lounge,Other Nightlife,3884,Howard Beach,11414,NY,United States,40.649237,-73.837212
4,Aldo's Pizzeria & Restaurant,Italian Restaurant,4871,Ozone Park,11417,NY,United States,40.672073,-73.843016
5,BJ's Restaurant & Brewhouse,American Restaurant,5689,Valley Stream,11581,NY,United States,40.663520,-73.725640
6,Port Royal Jamaican Restaurant,Caribbean Restaurant,3819,Jamaica,11435,NY,United States,40.686740,-73.794102
7,Ho Wan Restaurant,Chinese Restaurant,4911,Jamaica,11419,NY,United States,40.684857,-73.830962
8,The Nest Restaurant,Caribbean Restaurant,4998,Richmond Hill,11419,NY,United States,40.691697,-73.820287
9,Cascata Restaurant & Bar,Italian Restaurant,1201,Jamaica,11430,NY,United States,40.641733,-73.792545


In [51]:
if len(results_2['response']['venues']) == 0:
    trending_venues_map_2 = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    trending_venues_map_2 = folium.Map(location=[latitude_2, longitude_2], zoom_start=11) # generate map centred around Queens


    # add Queens as a red circle mark
    folium.features.CircleMarker(
        [latitude_2, longitude_2],
        radius=10,
        popup='Queens',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(trending_venues_map_2)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_2_df['location.lat'], trending_venues_2_df['location.lng'], trending_venues_2_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(trending_venues_map_2)
        
        


In [52]:
# display map
trending_venues_map_2